## Load Libraries

In [1]:
import tqdm
import os
import cv2
import imghdr
import image_to_numpy
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime
import itertools
from numpy.random import default_rng

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomContrast, RandomBrightness, Rescaling, Resizing
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, ReLU
from tensorflow.keras.metrics import F1Score, Precision, Recall, CategoricalAccuracy, BinaryAccuracy
from tensorboard.plugins.hparams import api as hp
import sklearn
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import StratifiedKFold

from time import perf_counter
import math

import neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback

## Get Dataset

In [2]:
labels = ['Not Road', 'Road']
X_train = []
y_train = []
y_label_idx = []
image_size = 256
for i in labels:
    folderPath = os.path.join('D:\\DATASETS\\FireBase-Data\\Unseen-Testing-Data-For-Binary', i)
    for j in os.listdir(folderPath):
        img = image_to_numpy.load_image_file(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_label_idx.append(labels.index(i))

In [3]:
X_train = np.array(X_train)
y_label_idx = np.array(y_label_idx)

X_train_scaled = X_train.astype(np.float32) / 255.0

## Evaluate Final-B4_0_1-converted  

In [4]:
model_path = "C:\\Users\\user\\Documents\\Machine-Learning-Hub-1\\Tensorflow\\kaggle_converted\\Final-B4_0_1-converted.keras"
sm = tf.keras.models.load_model(model_path)

In [5]:
pre = Precision()
rec = Recall()
c_acc = BinaryAccuracy()
f1_score = F1Score()


y = tf.keras.utils.to_categorical(y_label_idx)
for i in range(len(X_train_scaled)):
    ready_image = np.expand_dims(X_train_scaled[i], 0)
    yhat = sm.predict(ready_image, verbose=0)
    ytrue = np.expand_dims(y[i], 0)
    f1_score.update_state(ytrue, yhat)
    pre.update_state(ytrue, yhat)
    rec.update_state(ytrue, yhat)
    c_acc.update_state(ytrue, yhat)

# PRINTING RESULTS ===========================================
print("Binary Model Result")
print(f'Precision: {pre.result().numpy()}  -  Recall: {rec.result().numpy()}')
print(f'Accuracy: {c_acc.result().numpy()}')
print(f'F1Score: Not Road {f1_score.result().numpy()[0]} | Road {f1_score.result().numpy()[1]}')
    

Binary Model Result
Precision: 0.753731369972229  -  Recall: 0.753731369972229
Accuracy: 0.753731369972229
F1Score: Not Road 0.6206896901130676 | Road 0.8176795244216919


## Get Dataset

In [6]:
labels = ['Building','Plant', 'Road', 'Not Road']
X_train = []
y_train = []
y_label_idx = []
image_size = 256
for i in labels:
    
    folderPath = os.path.join("D:\\DATASETS\\FireBase-Data\\Unseen-Testing-Data-For-4_Class", i)
    if not os.path.exists(folderPath):
        continue
    for j in os.listdir(folderPath):
        img = image_to_numpy.load_image_file(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_label_idx.append(labels.index(i))

In [7]:
X_train = np.array(X_train)
y_label_idx = np.array(y_label_idx)

X_train_scaled = X_train.astype(np.float32) / 255.0

## Evaluate Model V4.0.1 (Deployed Model)

In [8]:
model_path = "C:\\Users\\user\\Documents\\Machine-Learning-Hub-1\\Exp-Flask-New-Thesis\\model\\RNR\\Old\\Model_V4-0-1.keras"
sm = tf.keras.models.load_model(model_path)

In [9]:
pre = Precision()
rec = Recall()
c_acc = BinaryAccuracy()
f1_score = F1Score()


y = tf.keras.utils.to_categorical(y_label_idx, 4)
for i in range(len(X_train_scaled)):
    ready_image = np.expand_dims(X_train_scaled[i], 0)
    yhat = sm.predict(ready_image, verbose=0)
    ytrue = np.expand_dims(y[i], 0)
    f1_score.update_state(ytrue, yhat)
    pre.update_state(ytrue, yhat)
    rec.update_state(ytrue, yhat)
    c_acc.update_state(ytrue, yhat)

# PRINTING RESULTS ===========================================
print("4-Class Deployed Model Result")
print(f'Precision: {pre.result().numpy()}  -  Recall: {rec.result().numpy()}')
print(f'Accuracy: {c_acc.result().numpy()}')
print(f'F1Score: Building {f1_score.result().numpy()[0]} | Plant {f1_score.result().numpy()[1]}')
print(f'F1Score: Road {f1_score.result().numpy()[2]} | Vehicle {f1_score.result().numpy()[3]}')
    

4-Class Deployed Model Result
Precision: 0.6880000233650208  -  Recall: 0.641791045665741
Accuracy: 0.8376865386962891
F1Score: Building 0.2068965584039688 | Plant 0.13333334028720856
F1Score: Road 0.813559353351593 | Vehicle 0.5106383562088013


## Get Dataset

In [10]:
labels = ['Building','Plant', 'Road', 'Not Road']
X_train = []
y_train = []
y_label_idx = []
image_size = 256
for i in labels:
    
    folderPath = os.path.join('D:\\DATASETS\\FireBase-Data\\Unseen-Testing-Data-For-4_Class', i)
    if not os.path.exists(folderPath):
        continue
    for j in os.listdir(folderPath):
        img = image_to_numpy.load_image_file(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_label_idx.append(labels.index(i))

In [11]:
X_train = np.array(X_train)
y_label_idx = np.array(y_label_idx)

X_train_scaled = X_train.astype(np.float32) / 255.0

## Evaluate AdrKFold-Final-4_0_1-converted

In [12]:
model_path = "C:\\Users\\user\\Documents\\Machine-Learning-Hub-1\\Tensorflow\\kaggle_converted\\AdrKFold-Final-4_0_1-converted.keras"
sm = tf.keras.models.load_model(model_path)

In [13]:
pre = Precision()
rec = Recall()
c_acc = BinaryAccuracy()
f1_score = F1Score()


y = tf.keras.utils.to_categorical(y_label_idx, 4)
for i in range(len(X_train_scaled)):
    ready_image = np.expand_dims(X_train_scaled[i], 0)
    yhat = sm.predict(ready_image, verbose=0)
    ytrue = np.expand_dims(y[i], 0)
    f1_score.update_state(ytrue, yhat)
    pre.update_state(ytrue, yhat)
    rec.update_state(ytrue, yhat)
    c_acc.update_state(ytrue, yhat)

# PRINTING RESULTS ===========================================
print("4-Class Deployed Model Result")
print(f'Precision: {pre.result().numpy()}  -  Recall: {rec.result().numpy()}')
print(f'Accuracy: {c_acc.result().numpy()}')
print(f'F1Score: Building {f1_score.result().numpy()[0]} | Plant {f1_score.result().numpy()[1]}')
print(f'F1Score: Road {f1_score.result().numpy()[2]} | Vehicle {f1_score.result().numpy()[3]}')
    

4-Class Deployed Model Result
Precision: 0.6412213444709778  -  Recall: 0.6268656849861145
Accuracy: 0.8190298676490784
F1Score: Building 0.14814816415309906 | Plant 0.125
F1Score: Road 0.8196721076965332 | Vehicle 0.3809524178504944


## Get Dataset

In [14]:
labels = ['Building','Plant', 'Road', 'Not Road']
X_train = []
y_train = []
y_label_idx = []
image_size = 256
for i in labels:
    
    folderPath = os.path.join('D:\\DATASETS\\FireBase-Data\\Unseen-Testing-Data-For-4_Class', i)
    if not os.path.exists(folderPath):
        continue
    for j in os.listdir(folderPath):
        img = image_to_numpy.load_image_file(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_label_idx.append(labels.index(i))

In [15]:
X_train = np.array(X_train)
y_label_idx = np.array(y_label_idx)

X_train_scaled = X_train.astype(np.float32) / 255.0

## Evaluate PCNN0206_01B

In [16]:
model_path = "C:\\Users\\user\\Documents\\Machine-Learning-Hub-1\\Exp-Flask-New-Thesis\\model\\RNR\\PCNN0206_01B.keras"
sm = tf.keras.models.load_model(model_path)

In [17]:
pre = Precision()
rec = Recall()
c_acc = BinaryAccuracy()
f1_score = F1Score()


y = tf.keras.utils.to_categorical(y_label_idx, 4)
for i in range(len(X_train_scaled)):
    ready_image = np.expand_dims(X_train_scaled[i], 0)
    yhat = sm.predict(ready_image, verbose=0)
    ytrue = np.expand_dims(y[i], 0)
    f1_score.update_state(ytrue, yhat)
    pre.update_state(ytrue, yhat)
    rec.update_state(ytrue, yhat)
    c_acc.update_state(ytrue, yhat)

# PRINTING RESULTS ===========================================
print("4-Class Deployed Model Result")
print(f'Precision: {pre.result().numpy()}  -  Recall: {rec.result().numpy()}')
print(f'Accuracy: {c_acc.result().numpy()}')
print(f'F1Score: Building {f1_score.result().numpy()[0]} | Plant {f1_score.result().numpy()[1]}')
print(f'F1Score: Road {f1_score.result().numpy()[2]} | Vehicle {f1_score.result().numpy()[3]}')
    

4-Class Deployed Model Result
Precision: 0.5440000295639038  -  Recall: 0.5074626803398132
Accuracy: 0.77052241563797
F1Score: Building 0.1764705777168274 | Plant 0.10526315867900848
F1Score: Road 0.6967741847038269 | Vehicle 0.46666666865348816
